In [3]:
import keras
from keras.models import Model
from keras.layers import *
from keras.optimizers import *
from keras.applications import InceptionResNetV2
from keras.preprocessing import image
import keras.backend as K

In [2]:
import tensorflow as tf
import numpy as np
from LR_SGD import LR_SGD

In [ ]:
model = InceptionResNetV2(include_top=False, input_shape=(224,224,3), weights='imagenet', pooling='avg')

In [44]:
ip=Input((1536,))
W = K.variable(K.random_normal((1536,300)), name='Weight')
w_emb = Lambda(lambda x: K.dot(x, W), name='Embedding')(ip)
recon_ip = Lambda(lambda x: K.dot(x, K.transpose(W)), name='Reconstructed_Input')(w_emb)

In [45]:
top_model= Model(inputs=ip, outputs=[w_emb, recon_ip])
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1536)              0         
_________________________________________________________________
Embedding (Lambda)           (None, 300)               0         
_________________________________________________________________
Reconstructed_Input (Lambda) (None, 1536)              0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [46]:
top_model.output_shape

[(None, 300), (None, 1536)]

In [8]:
seen_embeddings = np.load('../emeddings/seen_glove6B_embeddings.npy')

In [47]:
class_embeddings = K.constant(seen_embeddings)

def cos_fn(d, e):
    f = tf.nn.l2_normalize(d, axis=-1)
    g = tf.nn.l2_normalize(e, axis=-1)
    return tf.reduce_sum(tf.multiply(f, g), axis=-1)

def loss_fn(y_pred, true_cls_emb):
    pred_cls_emb = y_pred[0]
    a = cos_fn(pred_cls_emb, class_embeddings) - cos_fn(pred_cls_emb, true_cls_emb) + 1
    recon_loss = 1e-3*K.sum(K.square(y_pred[1] - ip), axis=-1)
    return -1 + K.sum(K.maximum(a, 0), axis=-1) + recon_loss

In [6]:
LR_mult_dict = {
    'Embedding' : 1,
    'Reconstructed_Input' : 0.01
}
base_lr = 0.001
momentum = 0.9

In [48]:
opt = LR_SGD(lr=base_lr, momentum=momentum, decay=0.0, multipliers=LR_mult_dict)
top_model.compile(opt, loss=loss_fn, metrics=['accuracy'])